In [8]:
from beanie import Document, Indexed, init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from pydantic import BaseModel

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
class Category(BaseModel):
    name: str
    description: str

    class Settings:
        projection = {"name": "$category.name", "description": "$category.description"}


class Product(Document):  # This is the model
    category: Category
    price: Indexed(float)

    class Settings:
        name = "products"


client = AsyncIOMotorClient("mongodb://localhost:27017")

# Initialize beanie with the Sample document class and a database
await init_beanie(database=client.db_name, document_models=[Product])

In [65]:
client.db_name

AsyncIOMotorDatabase(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=False, driver=DriverInfo(name='Motor', version='3.7.0', platform='asyncio')), 'db_name'))

In [70]:
chocolate_category = Category(name="Chocolate", description="A preparation of roasted and ground cacao seeds.")
candy_category = Category(name="Candy", description="A preparation of roasted and ground cacao seeds.")
tonybar = Product(price=5.95, category=chocolate_category)
marsbar = Product(price=1, category=chocolate_category)
candybar = Product(price=2, category=candy_category)

In [71]:
await Product.insert_many([tonybar, marsbar, candybar])

InsertManyResult([ObjectId('67a253a3cec60c8c4edbd16d'), ObjectId('67a253a3cec60c8c4edbd16e'), ObjectId('67a253a3cec60c8c4edbd16f')], acknowledged=True)

In [72]:
await Product.find().to_list()

[Product(id=ObjectId('67a252e8cec60c8c4edbd169'), revision_id=None, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'), price=5.95),
 Product(id=ObjectId('67a252e8cec60c8c4edbd16a'), revision_id=None, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'), price=1.0),
 Product(id=ObjectId('67a253a3cec60c8c4edbd16d'), revision_id=None, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'), price=5.95),
 Product(id=ObjectId('67a253a3cec60c8c4edbd16e'), revision_id=None, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'), price=1.0),
 Product(id=ObjectId('67a253a3cec60c8c4edbd16f'), revision_id=None, category=Category(name='Candy', description='A preparation of roasted and ground cacao seeds.'), price=2.0)]

In [73]:
stuff = await Product.find().project(Category).to_list()
stuff

[Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'),
 Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'),
 Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'),
 Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'),
 Category(name='Candy', description='A preparation of roasted and ground cacao seeds.')]

In [69]:
chocolates

[Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'),
 Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.')]